In [1]:
import sys
sys.path.append('../')

In [2]:
from pathlib import Path

import pandas as pd

from legacy_code_assistant.knowledge_base.knowledge_builder import KnowledgeBaseBuilder
from legacy_code_assistant.knowledge_base.knowledge_builder import CodeAnalyzer

In [3]:
path = Path() / '..' / '..' / 'Django-School-Management-System'
# path = Path() / '..' / 'test_code'


In [4]:
paths = list(path.rglob('**/*.py'))
ca = CodeAnalyzer(paths)
results = ca.analyze()
print(len(results))
results

218


[{'docstring': None,
  'code': 'def main():\n    os.environ.setdefault(\'DJANGO_SETTINGS_MODULE\', \'school_app.settings\')\n    try:\n        from django.core.management import execute_from_command_line\n    except ImportError as exc:\n        raise ImportError(\n            "Couldn\'t import Django. Are you sure it\'s installed and available on your PYTHONPATH environment variable? Did you forget to activate a virtual environment?"\n            ) from exc\n    execute_from_command_line(sys.argv)\n',
  'code_start_line': 7,
  'code_end_line': 17,
  'file': '../../Django-School-Management-System/manage.py',
  'module': 'manage',
  'name': 'main',
  'parent': 'manage',
  'type': 'function'},
 {'docstring': "Django's command-line utility for administrative tasks.",
  'code': '"""Django\'s command-line utility for administrative tasks."""\nimport os\nimport sys\n\n\ndef main():\n    os.environ.setdefault(\'DJANGO_SETTINGS_MODULE\', \'school_app.settings\')\n    try:\n        from django.c

In [5]:
df = pd.DataFrame(results)
df['len'] = df['code'].apply(lambda x: len(x))
df = df.sort_values('len', ascending=False)
df.head(10)

,docstring,code,code_start_line,code_end_line,file,module,name,parent,type,len
169,None,from django.contrib import messages\nfrom djan...,0,NaN,../../Django-School-Management-System/apps/cor...,views,views,NaN,module,9495
103,None,from django.contrib.auth.decorators import log...,0,NaN,../../Django-School-Management-System/apps/fin...,views,views,NaN,module,4093
33,None,from django.contrib import messages\nfrom djan...,0,NaN,../../Django-School-Management-System/apps/res...,views,views,NaN,module,3621
214,Django settings for school_app project.\n\nGen...,"""""""\nDjango settings for school_app project.\n...",0,NaN,../../Django-School-Management-System/school_a...,settings,settings,NaN,module,3281
62,None,import csv\nfrom django.contrib.auth.mixins im...,0,NaN,../../Django-School-Management-System/apps/stu...,views,views,NaN,module,3081
73,None,import csv\nimport os\nfrom io import StringIO...,0,NaN,../../Django-School-Management-System/apps/stu...,signals,signals,NaN,module,2715
123,None,from django.db import models\nfrom django.urls...,0,NaN,../../Django-School-Management-System/apps/fin...,models,models,NaN,module,2081
126,None,"from django.db import migrations, models\nimpo...",0,NaN,../../Django-School-Management-System/apps/fin...,0001_initial,0001_initial,NaN,module,2012
69,None,"@receiver(post_save, sender=StudentBulkUpload)...",14,68.0,../../Django-School-Management-System/apps/stu...,signals,create_bulk_student,signals,function,2003
84,None,"from django.db import migrations, models\nimpo...",0,NaN,../../Django-School-Management-System/apps/stu...,0001_initial,0001_initial,NaN,module,1911


<h1>Generating docstrings / descriptions</h2>

In [6]:
import yaml
import os

# get credentials from yaml
# credentialsy sa wrzucone na discordzie

with open("credentials.yaml", "r") as f:
    credentials = yaml.load(f, Loader=yaml.FullLoader)

os.environ["AZURE_OPENAI_ENDPOINT"] = credentials['AZURE_OPENAI_ENDPOINT']
os.environ["AZURE_OPENAI_API_KEY"] = credentials['AZURE_OPENAI_API_KEY']

In [7]:
from tqdm import tqdm
import re

from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage, AIMessage
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate


In [8]:
model = AzureChatOpenAI(
    openai_api_version="2023-05-15",
    azure_deployment=credentials['Deployment'],
)

In [9]:
df_docstringable = df.loc[df['type'] != 'module']
print(len(df_docstringable), '/', len(df))
df_docstringable.head()

151 / 218


,docstring,code,code_start_line,code_end_line,file,module,name,parent,type,len
69,None,"@receiver(post_save, sender=StudentBulkUpload)...",14,68.0,../../Django-School-Management-System/apps/stu...,signals,create_bulk_student,signals,function,2003
125,None,class Migration(migrations.Migration):\n in...,8,119.0,../../Django-School-Management-System/apps/fin...,0001_initial,Migration,NaN,class,1907
30,None,@login_required\ndef create_result(request):\n...,14,69.0,../../Django-School-Management-System/apps/res...,views,create_result,views,function,1812
83,None,class Migration(migrations.Migration):\n in...,8,91.0,../../Django-School-Management-System/apps/stu...,0001_initial,Migration,NaN,class,1806
204,None,class Migration(migrations.Migration):\n in...,6,105.0,../../Django-School-Management-System/apps/cor...,0001_initial,Migration,NaN,class,1590


In [10]:
df_docstringable.isna().sum()

docstring          137
code                 0
code_start_line      0
code_end_line        0
file                 0
module               0
name                 0
parent              83
type                 0
len                  0
dtype: int64

In [11]:
prompt = '''
Given the code of the {type} below, write a docstring for it.
Do not write anything other than the docstring, docstring should be the only output.
Start your answer with ```python\n"""\n and end it with """\n```
\n\n{code}\n\n
'''

# prompt_ai = '''
# Docstring:\n\n
# ```python\n"""\n
# '''

chat_prompt_template = ChatPromptTemplate.from_template(prompt)
# ai_message = AIMessage(content=prompt_ai)

In [12]:
example = df_docstringable.iloc[0]
chat_prompt_template.format_prompt(type=example['type'], code=example['code'])

ChatPromptValue(messages=[HumanMessage(content='\nGiven the code of the function below, write a docstring for it.\nDo not write anything other than the docstring, docstring should be the only output.\nStart your answer with ```python\n"""\n and end it with """\n```\n\n\n@receiver(post_save, sender=StudentBulkUpload)\ndef create_bulk_student(sender, created, instance, *args, **kwargs):\n    if created:\n        opened = StringIO(instance.csv_file.read().decode())\n        reading = csv.DictReader(opened, delimiter=\',\')\n        students = []\n        for row in reading:\n            if \'registration_number\' in row and row[\'registration_number\']:\n                reg = row[\'registration_number\']\n                surname = row[\'surname\'] if \'surname\' in row and row[\'surname\'\n                    ] else \'\'\n                firstname = row[\'firstname\'] if \'firstname\' in row and row[\n                    \'firstname\'] else \'\'\n                other_names = row[\'other_

In [13]:
prompts = [chat_prompt_template.format_prompt(type=example['type'], code=example['code']) for _, example in df_docstringable.iterrows()]

In [17]:
docstrings = []

for prompt in tqdm(prompts):
    result = model(prompt.to_messages()).content
    result = re.sub('^\n*```\n*(python)\n*(""")?\n*', '', result)
    result = re.sub('\n*(""")?\n*```\n*$', '', result)
    docstrings.append(result)

100%|██████████| 151/151 [21:32<00:00,  8.56s/it]


In [19]:
len(docstrings)

151

In [22]:
df_docstringable.loc[:, 'generated_docstring'] = docstrings
df_docstringable.head()

,docstring,code,code_start_line,code_end_line,file,module,name,parent,type,len,generated_docstring
69,None,"@receiver(post_save, sender=StudentBulkUpload)...",14,68.0,../../Django-School-Management-System/apps/stu...,signals,create_bulk_student,signals,function,2003,Signal receiver for the post_save event of the...
125,None,class Migration(migrations.Migration):\n in...,8,119.0,../../Django-School-Management-System/apps/fin...,0001_initial,Migration,NaN,class,1907,A Django database migration for the creation o...
30,None,@login_required\ndef create_result(request):\n...,14,69.0,../../Django-School-Management-System/apps/res...,views,create_result,views,function,1812,Processes the creation of academic results for...
83,None,class Migration(migrations.Migration):\n in...,8,91.0,../../Django-School-Management-System/apps/stu...,0001_initial,Migration,NaN,class,1806,This migration class defines the initial datab...
204,None,class Migration(migrations.Migration):\n in...,6,105.0,../../Django-School-Management-System/apps/cor...,0001_initial,Migration,NaN,class,1590,A Django database migration subclass for initi...


In [23]:
df.to_csv('knowledge_base.csv', index=False)

In [24]:
df_docstringable.to_csv('functions_with_generated_docstrings.csv', index=False)

<h1>Make vectorstore based on code</h1>

In [6]:
from legacy_code_assistant.knowledge_base.knowledge_builder import KnowledgeBaseBuilder
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter #-> to splits data to chunks




In [8]:
kbb = KnowledgeBaseBuilder(index_name='code_based_index')
kbb.initialize_faiss_based_on_df(df, text_column='code')

kbb.search('endpoint')

No sentence-transformers model found with name /home/laszer/.cache/torch/sentence_transformers/microsoft_codebert-base. Creating a new one with MEAN pooling.


[(Document(page_content='from django.test import TestCase', metadata={'docstring': None, 'code_start_line': 0, 'code_end_line': nan, 'file': '../../Django-School-Management-System/apps/staffs/tests.py', 'module': 'tests', 'name': 'tests', 'parent': nan, 'type': 'module', 'len': 33}),
  58.918568)]

In [9]:
kbb.save_index()

<h1>Make vectorstore based on generated docstrings</h1>

In [10]:
df = pd.read_csv('functions_with_generated_docstrings.csv')

In [ ]:
kbb = KnowledgeBaseBuilder(index_name='code_based_index', model_name='facebook/bart-large-cnn')
kbb.initialize_faiss_based_on_df(df, text_column='code')

kbb.search('endpoint')